In [1]:
!pip install spotipy --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import os

In [3]:
os.environ['SPOTIPY_CLIENT_ID'] = '7c019efe64ad4f898c264803996b4863'
os.environ['SPOTIPY_CLIENT_SECRET'] = '261ba5610d1a4376924fa21d08468677'
os.environ['SPOTIPY_REDIRECT_URI'] = 'https://localhost:8888/callback'

In [4]:
#Create Spotify object
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [5]:
def colect(playlist_URI):
  pl_id = sp.playlist(playlist_URI)['id']
  pl_name = sp.playlist(playlist_URI)['name']
  results = sp.playlist_tracks(playlist_URI)
  pl = results['items']
  while results['next']:
    results = sp.next(results)
    pl.extend(results['items'])
  print(type(pl), len(pl))
  items = []
  one_item = []
  removal = ['type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature']
  
  for item in pl:
    one_item.clear()
    one_item.append(item['track']['id'])
    one_item.append(item['track']['name'])
    one_item.append(item['track']['artists'][0]['id']) #tem muitos aaaaaa
    one_item.append(item['track']['artists'][0]['name'])
    one_item.append(item['track']['album']['id'])
    one_item.append(item['track']['duration_ms'])
    one_item.append(item['track']['album']['release_date'])
    one_item.append(item['track']['popularity'])
    af = sp.audio_features([item['track']['id']])[0]
    [af.pop(key) for key in removal]
    for key in af:
      one_item.append(af[key])
    one_item.append((item['track']['duration_ms'])/60000)
    one_item.append(pl_id)
    one_item.append(pl_name)
    release_year = item['track']['album']['release_date'][0:4]
    one_item.append(release_year)
    items.append(list(one_item))

  db = pd.DataFrame(items, columns=['track_id', 'track_name', 'artist_id', 'artist_name', 'album_id', 'duration', 'release_date', 'popularity', 'danceability', 'energy', 'key', 'loudness' ,'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_mins', 'playlist_id', 'playlist_name', 'release_year'])
  return db


In [6]:
def cleaning(db, nome):
  print("Número de registros no dataset:", len(db))
  db.dropna(inplace=True)
  db.reset_index(drop=True, inplace=True)

  print("Número de registros no dataset após remover as linhas com dados faltantes:", len(db))
  print(db.columns)
  duplicates = db[db.duplicated(keep='last')]

  print("Número de registros duplicados:", len(duplicates))
  print(duplicates)
  db.drop_duplicates(inplace=True)
  db.reset_index(drop=True, inplace=True)

  print("Número de registros sem as duplicatas:", len(db))
  columns = ['track_name', 'artist_name', 'album_id', 'artist_id', 'playlist_id', 'playlist_name', 'release_date']
  numeric_columns = db.drop(columns, axis=1)
  numeric_columns.head()
  db.to_csv('../data/{}.csv'.format(nome), index= False)
  numeric_columns.to_csv('../data/{}_numeric.csv'.format(nome), index= False)


In [9]:
playlist_link = "https://open.spotify.com/playlist/50whMXcsw77NidwaF9hMcb?si=-b37hncZQnCyHAwJFbm31Q&utm_source=whatsapp"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
testdb = colect(playlist_URI)
cleaning(testdb, "Teste")

<class 'list'> 2500


,track_id,track_name,artist_id,artist_name,album_id,duration,release_date,popularity,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,playlist_id,playlist_name,release_year
0,2P1mLMA0W0wpLPPlYIhBa9,Satellite,445d0aCtpY1jmpLO87SfNu,The Sweet Kill,3H7rNKPK3qmkKeBKDwrLo2,209176,2022-02-04,22,0.488,0.832,...,0.0418,0.01070,0.3950,0.112,0.422,107.015,3.486267,50whMXcsw77NidwaF9hMcb,GOTH ULTIMATE LIST | DARK WAVE/POST-PUNK/GOTHI...,2022
1,2Zh1LkF9VziQVUjzIFlyi9,Prayers For Rain - Remastered,7bu3H8JO7d0UbMoVzbo70s,The Cure,0H6TddUF2M63ZSHGvhk5yy,367426,1989-05-02,41,0.565,0.680,...,0.0269,0.00460,0.0936,0.076,0.121,116.973,6.123767,50whMXcsw77NidwaF9hMcb,GOTH ULTIMATE LIST | DARK WAVE/POST-PUNK/GOTHI...,1989
2,1390XaOMSPI8rJUyKiL18Y,Give Me Light (Single Version),3rGQvGgLMAJW4Zz17bcpGZ,Camlann,0brehNXlxLl6hZfzImxAcs,268250,2021-03-12,3,0.566,0.742,...,0.0336,0.00161,0.2380,0.107,0.140,119.952,4.470833,50whMXcsw77NidwaF9hMcb,GOTH ULTIMATE LIST | DARK WAVE/POST-PUNK/GOTHI...,2021
3,1bqerS49YfUgCimJQJPUsC,Holding On,3pIO4LI39vyMYT9cou1B7j,This Cold Night,4efXao3lMzP4AWTn94pFL5,201504,2016-01-02,27,0.598,0.680,...,0.0507,0.41300,0.8940,0.111,0.426,156.018,3.358400,50whMXcsw77NidwaF9hMcb,GOTH ULTIMATE LIST | DARK WAVE/POST-PUNK/GOTHI...,2016
4,5trdk633doJmre0SiVIZwV,New Dawn Fades,2jpwQwRV1LWAGIPhlnvsXE,Dragon Rapid,44qFcr69JqDgrrqHQleHq6,292480,2022-10-28,12,0.516,0.906,...,0.0347,0.03240,0.5830,0.132,0.275,155.996,4.874667,50whMXcsw77NidwaF9hMcb,GOTH ULTIMATE LIST | DARK WAVE/POST-PUNK/GOTHI...,2022


In [8]:
#Para coletar uma playlist faça:
# playlist_link = Link da playlist aqui!
# playlist_URI = playlist_link.split("/")[-1].split("?")[0]
# db = colect(playlist_URI)
# db.head()
# cleaning(testdb, Nome da playlist aqui!)